In [1]:
import pandas as pd
import altair as alt
import datetime
import sqlite3
from mlrepricer import helper, setup
from mlrepricer.oldsql import schemas, database
%alias_magic t timeit
import os

alt.data_transformers.enable('default', max_rows=1000000)

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


DataTransformerRegistry.enable('default')

### load the data

In [2]:
t = schemas.pricemonitor(database.SQLite)()  # tableobject
df = pd.read_sql_query(f'SELECT * FROM {t.table}', t.conn, parse_dates=[t.eventdate], index_col='ID')

OperationalError: (sqlite3.OperationalError) unable to open database file (Background on this error at: http://sqlalche.me/e/e3q8)

### example analysis

In [ ]:
def what():
    sub = df[(df.asin=='B01H0GZ2M2') & (df.isbuyboxwinner==1)]
    sub.index = sub.time_changed
    primesub = sub.loc[sub.isprime==1].copy()
    primesub.sort_index(inplace=True)
    print(len(primesub))
    primesub.loc[primesub.index.duplicated(keep=False)]
    len(primesub)

In [ ]:
# a speaking multiindex, a replacement to the messageid found in the sqs queue
df['messageid'] = df['asin']+'_'+df['time_changed'].dt.strftime('%Y-%m-%d %H:%M:%S.%f')

# latest shit we need at all?
latest_df = df.groupby('asin')['time_changed'].max()
lookup_snapshot = latest_df.index+'_'+latest_df.dt.strftime('%Y-%m-%d %H:%M:%S.%f')
snapshot = df.loc[df['messageid'].isin(lookup_snapshot)]

In [ ]:
# We sort the asins by the number of messearuments we got for the asin
sortedgroup = df.groupby(['asin'])['time_changed'].count().sort_values(ascending=False)
asins = list(sortedgroup.index)[0:10]
asin = [list(sortedgroup.index)[0]]

In [ ]:
asin

In [ ]:
examplelistings = df[df['asin'].isin(asins)]
examplelisting = df[df['asin'].isin(asin)]

In [56]:
examplelisting.price.max()

16993.0

In [47]:
from altair import *
oneasin = alt.Chart(examplelisting).mark_line(opacity=0.9).encode(
    alt.X('time_changed'),
    y='min(price)',
    color='sellerid:N',)

In [ ]:
#a.savechart('chart.svg')

In [ ]:
alt.Chart(examplelistings).mark_line().encode(
    x='time_changed:T',
    y='min(price)',
    color='asin')

In [ ]:
winner = examplelistings[examplelistings['isbuyboxwinner'] == 1]

In [ ]:
alt.Chart(winner).mark_line().encode(
    x='time_changed:T',
    y='min(price)',
    color='asin')